In [1]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

In [5]:
# Load Yaml file and load yolo model
with open('data.yaml', mode='r') as f:
    data_yaml = yaml.load(f, Loader=SafeLoader)
labels = data_yaml['names']

print(labels)

['person', 'car', 'chair', 'bottle', 'pottedplant', 'bird', 'dog', 'sofa', 'bicycle', 'horse', 'boat', 'motorbike', 'cat', 'tvmonitor', 'cow', 'sheep', 'aeroplane', 'train', 'diningtable', 'bus']


In [13]:
# os.chdir('./YOLO_Obj_Detection/2_Predictions')

In [14]:
yolo = cv2.dnn.readNetFromONNX('./Model/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [23]:
# Load the image
img = cv2.imread('./street_image.jpg')
image = img.copy()
row, col, d = image.shape

# Get the yolo predictions from the image
# Step 1: Convert the image to square shape
max_rc = max(row, col)
input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
input_image[0:row,0:col] = image

# Step 2: Get predictions from square array
INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO,INPUT_WH_YOLO), swapRB=True, crop=False)
yolo.setInput(blob)
preds = yolo.forward() #detections or predictions from yolo


In [24]:
print(preds.shape)

(1, 25200, 25)


In [36]:
# Non maximum suppression
# Step 1: filter detections based on confidence score (0.4) and probability score (0.25)
detections = preds[0]
boxes = []
confidences = []
classes = []

# width and height of the input image
image_w, image_h = input_image.shape[:2]
x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4] #confidence of detection of an object
    if confidence>0.4:
        class_score = row[5:].max() # maximum probability from 20 objects
        class_id = row[5:].argmax() # get the index position at which max probability occurs

        if class_score>0.25:
            cx,cy,w,h = row[0:4]
            # constructing bounding box
            left = int((cx-0.5*w)*x_factor)
            top = int((cy-0.5*h)*y_factor)
            width = int(w*x_factor)
            height = int(h*y_factor)

            box = np.array([left, top, width, height])
            
            # append values into list
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)

# clean
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

# NMS
index = np.array(cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.25, 0.4)).flatten()

In [38]:
# These many number of objects are identified from the image
len(index)

0

In [40]:
# Draw the bounding boxes

for ind in index:
    # extract bounding boxes
    x,y,w,h = boxes_np[ind]
    bb_conf = int(confidences_np[ind]*100)
    classes_id = classes[ind]
    class_name = labels[classes_id]
    
    text = f'{class_name}: {bb_conf}%'
    cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)
    cv2.rectangle(image, (x,y-30), (x+w,y), (255,255,255), -1)
    cv2.putText(image, text, (x,y-10), FONT_HERSHEY_PLAIN, 0.7, (0,0,0), 1)


In [41]:
cv2.imshow('original', img)
cv2.imshow('yolo_predictions', image)

cv2.waitKey(0)
cv2.destroyAllWindows()